In [14]:
# imports
# data collection
import feedparser
import newspaper
import pandas as pd
from datetime import datetime
import time
import sys
import requests
from bs4 import BeautifulSoup

# data processing
import torch
import numpy as np
from langchain_community.vectorstores import FAISS  # For storing and retrieving embeddings using the FAISS library
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import joblib # For saving models

#Cluster Classifier & Interpretation
import shap
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# Fraser Data Collection

def scrape_financial_crisis_timeline(url):
    """Scrapes date, title, description, and link info from the timeline."""
    try:
        # 1. Download the HTML content
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

        # 2. Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # 3. Find the main container element
        # Your target container is class="timeline-events clusterize-scroll" and id="list-container"
        timeline_container = soup.find('div', id='list-container')
        
        if not timeline_container:
            print("Error: Main timeline container not found.")
            return []

        # 4. Find all individual event rows
        # The articles are inside <div class="row event-row active">
        event_rows = timeline_container.find_all('div', class_='event-row')

        data = []
        for row in event_rows:
            # 5. Extract data points using the specific classes
            
            # Date and Source/Title: <h2 class="list-item">
            header_element = row.find('h2', class_='list-item')
            header_text = header_element.text.strip() if header_element else 'N/A'
            
            # Description/Summary: <p class="list-item">
            summary_element = row.find('p', class_='list-item')
            summary = summary_element.text.strip() if summary_element else 'N/A'
            
            # Associated Link: <ul><li><a href="..." class="list-item">
            link_element = row.find('a', class_='list-item')
            
            link_title = link_element.text.strip() if link_element else 'N/A'
            link_url = link_element['href'] if link_element else 'N/A'
            
            # Prepend the base URL if the link is relative
            if link_url != 'N/A' and link_url.startswith('/'):
                link_url = 'https://fraser.stlouisfed.org' + link_url
            
            # Split the header into Date and Source
            if '|' in header_text:
                date, source = [part.strip() for part in header_text.split('|', 1)]
            else:
                date = header_text
                source = 'N/A'

            data.append({
                'Date': date,
                'Source': source,
                'Summary': summary,
                'Associated Link Title': link_title,
                'Associated Link URL': link_url
            })

        return data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# --- Execution ---
URL = "https://fraser.stlouisfed.org/timeline/financial-crisis"
fraser_timeline_data = scrape_financial_crisis_timeline(URL)

# Output the results (first 5 entries)
if fraser_timeline_data:
    df = pd.DataFrame(fraser_timeline_data)
    print(f"Scraped {len(df)} events.")
    print("\n--- First 5 Scraped Events ---")
    print(df.head().to_markdown(index=False))
else:
    print("Failed to scrape data.")


Scraped 305 events.

--- First 5 Scraped Events ---
| Date              | Source                                        | Summary                                                                                                                                                          | Associated Link Title                                                                                                                                                                                                                  | Associated Link URL                                                              |
|:------------------|:----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# A specific model for generating embeddings, distinct from the sentiment classifier
EMBEDDING_MODEL_NAME = "ProsusAI/finbert" 

def get_finbert_sentence_embeddings(texts, model_name=EMBEDDING_MODEL_NAME):
    """
    Loads FinBERT, tokenizes texts, and extracts the [CLS] token embedding 
    as the sentence representation.
    """
    # 1. Load Tokenizer and Model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Using AutoModel for embeddings (not AutoModelForSequenceClassification, which is for sentiment classification)
    model = AutoModel.from_pretrained(model_name)
    
    # Check for GPU and move model if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print(f"\n--- Generating FinBERT Embeddings on {device} ---")
    
    embeddings = []
    
    for text in texts:
        # 2. Tokenize the text
        # BERT models have a max sequence length (typically 512). 
        # The 'full_text' needs truncation or splitting (advanced step).
        inputs = tokenizer(
            text, 
            return_tensors="pt", 
            padding='max_length', 
            truncation=True, 
            max_length=512
        ).to(device)

        with torch.no_grad():
            # 3. Get model outputs
            outputs = model(**inputs)
            # The last hidden state contains the final embeddings for all tokens
            last_hidden_state = outputs.last_hidden_state
            
            # 4. Extract the [CLS] token vector as the sentence embedding
            # [CLS] token is at index 0, and we squeeze to remove the batch dimension
            cls_embedding = last_hidden_state[:, 0, :].squeeze().cpu().numpy()
            embeddings.append(cls_embedding)

    return np.array(embeddings)

if not df.empty:
    # Use the full_text column from the DataFrame created in Step 1
    texts_to_embed = df['Summary'].tolist()

    # Get the 768-dimensional embeddings
    finbert_embeddings = get_finbert_sentence_embeddings(texts_to_embed)

    # Add embeddings to the DataFrame for the next pipeline step
    df['finbert_embedding'] = list(finbert_embeddings)

    print("\n--- Embeddings Generated ---")
    print(f"Shape of Embeddings: {finbert_embeddings.shape}")
    print("DataFrame with embeddings ready for PCA/Clustering.")
else:
    print("No articles to process. Please check data acquisition step.")


--- Generating FinBERT Embeddings on cpu ---

--- Embeddings Generated ---
Shape of Embeddings: (305, 768)
DataFrame with embeddings ready for PCA/Clustering.


In [9]:
# PCA
embeddings = np.array(df['finbert_embedding'].tolist())
n_components = 100  # Target reduced dimension
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)
print(f"Scaled Embeddings Shape: {scaled_embeddings.shape}")

# 2. Dimensionality Reduction (PCA)
# Target: Reduce from 768-dim to a more manageable size (e.g., 100)
pca = PCA(n_components=n_components, random_state=42)
reduced_data = pca.fit_transform(scaled_embeddings)
print(f"PCA Reduced Data Shape: {reduced_data.shape}")
print(reduced_data[0])  # Print the first reduced vector

Scaled Embeddings Shape: (305, 768)
PCA Reduced Data Shape: (305, 100)
[-14.84868      6.1126013    3.4305367    5.76674     -9.525345
  -1.1116639    5.3272      -7.2235284   -0.02011316   1.2409043
   3.5410328   -0.6319603   -9.017167    -1.624022    -1.6866331
   0.5924423   -1.8345335   -0.09928001   0.26945448   7.2866435
  -0.7791316    1.4217315    7.994451     0.43119586  -1.099438
  -3.9576192    6.4014726    5.5783834    3.2479494    0.608338
   2.9499784    3.0143456   -0.07458016   4.8942504    0.07216936
   2.028043    -1.4730884   -0.59757      1.3931813    1.0009661
  -0.34108952   1.0862185   -1.6466125    1.3988231    0.7174072
   0.3173057    0.4327445    0.42597026  -0.9464779   -0.84037995
  -0.5054373   -2.5273292    0.693739     1.0381192   -0.58131975
   1.1044999    0.7060808   -1.6274889   -0.36401194  -0.5489241
   1.8548692    2.7607353    4.1266346    1.2411493    0.5907885
  -1.4627923    1.5553414    1.3721336   -1.4760609   -1.0088372
  -1.3397287   -0.2

In [10]:
# Get optimal amount of clusters using silhouette score
reduced_data = reduced_data.astype(np.float64)
range_n_clusters = list(range(2, 11))
best_n_clusters = 0
best_silhouette_score = -1
for n_clusters in range_n_clusters:
    gmm = GaussianMixture(n_components=n_clusters, random_state=42, reg_covar=1e-5)
    cluster_labels = gmm.fit_predict(reduced_data)
    
    silhouette_avg = silhouette_score(reduced_data, cluster_labels)
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")
    
    if silhouette_avg > best_silhouette_score:
        best_silhouette_score = silhouette_avg
        best_n_clusters = n_clusters

print(f"\nBest number of clusters by silhouette score: {best_n_clusters} with a score of {best_silhouette_score}")

For n_clusters = 2, the average silhouette_score is : 0.27166562223966845
For n_clusters = 3, the average silhouette_score is : 0.19192472920384226
For n_clusters = 4, the average silhouette_score is : 0.183505553280819
For n_clusters = 5, the average silhouette_score is : 0.14040711896718577
For n_clusters = 6, the average silhouette_score is : 0.15328536003952573
For n_clusters = 7, the average silhouette_score is : 0.14642039211012797
For n_clusters = 8, the average silhouette_score is : 0.1524589730811093
For n_clusters = 9, the average silhouette_score is : 0.14855807291520234
For n_clusters = 10, the average silhouette_score is : 0.14635362145389713

Best number of clusters by silhouette score: 2 with a score of 0.27166562223966845


In [11]:
# Clustering with the best number of clusters
n_clusters = best_n_clusters
gmm = GaussianMixture(n_components=n_clusters, random_state=42, covariance_type='full')
gmm_labels = gmm.fit_predict(reduced_data)
df['Cluster'] = gmm_labels
print("\n--- Clustering Completed ---")
print(df[['Date', 'Source', 'Cluster']].head())


--- Clustering Completed ---
                Date                                         Source  Cluster
0  February 27, 2007                      Freddie Mac Press Release        0
1      April 2, 2007  SEC Filing: New Century Financial Corporation        0
2       June 1, 2007                        Congressional Testimony        0
3       June 7, 2007              Bear Stearns Suspends Redemptions        0
4      June 28, 2007                  Federal Reserve Press Release        0


In [16]:
def interpret_clusters_with_shap(reduced_data,
                                 cluster_labels,
                                 df,
                                 pca_model,
                                 scaler=None,
                                 top_pcs=10,
                                 top_embeddings=20,
                                 n_prototypes=3,
                                 shap_sample=200,
                                 random_state=42):
    """
    Improved cluster interpretation pipeline.
    Args:
      reduced_data: ndarray (n_samples, n_pca_components) -- PCA-transformed data used for clustering
      cluster_labels: ndarray (n_samples,) -- cluster ids from GMM/other
      df: pandas.DataFrame -- original dataframe containing text fields ('Summary' or 'full_text', 'title' optional)
      pca_model: fitted sklearn PCA instance used to produce reduced_data
      scaler: optional fitted scaler applied before PCA (if used). If not None, used for reverse mapping comments.
      top_pcs: number of top PCA components to report per cluster
      top_embeddings: number of original embedding dimensions to surface when mapping back
      n_prototypes: number of prototype documents to show per cluster
      shap_sample: max number of rows used by SHAP explainer for speed (Kernel explainer avoided)
      Returns:
        results dict with per-cluster SHAP importances, mapped original-dim importances,
        prototype indices and keywords.
    Notes:
      - This function trains a RandomForest surrogate (works well with TreeExplainer for SHAP).
      - Reverse mapping from PCA -> original embedding dims is approximate and shows which
        embedding-dimensions (e.g., FinBERT hidden dims) matter most. Mapping embedding dims
        to tokens requires further steps (token-level activations / nearest-neighbor in token embedding space).
    """
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    import shap
    import matplotlib.pyplot as plt

    np.random.seed(random_state)

    # Validate inputs
    n_samples, n_pcs = reduced_data.shape
    unique_clusters = np.unique(cluster_labels)
    n_clusters = unique_clusters.size
    assert len(cluster_labels) == n_samples, "cluster_labels length mismatch"

    # 1) Train surrogate classifier (Random Forest) to predict cluster from PCA features
    X_train, X_test, y_train, y_test = train_test_split(
        reduced_data, cluster_labels, test_size=0.2, random_state=random_state, stratify=cluster_labels
    )

    clf = RandomForestClassifier(n_estimators=200, random_state=random_state, n_jobs=-1)
    clf.fit(X_train, y_train)

    print("\nSurrogate classifier trained. Test accuracy: {:.3f}".format(clf.score(X_test, y_test)))

    # 2) SHAP explanation using TreeExplainer (fast for tree models)
    # Sample background for SHAP if dataset is large
    background = X_train
    if background.shape[0] > shap_sample:
        idx = np.random.choice(background.shape[0], shap_sample, replace=False)
        background = background[idx]

    explainer = shap.TreeExplainer(clf, data=background, model_output='probability')
    # Use a reasonable test subset for SHAP values (speed)
    shap_X = X_test
    if shap_X.shape[0] > shap_sample:
        idx = np.random.choice(shap_X.shape[0], shap_sample, replace=False)
        shap_X = shap_X[idx]

    shap_values = explainer.shap_values(shap_X)  # list length = n_classes, each (m, n_pcs)

    results = {
        'n_clusters': n_clusters,
        'cluster_summaries': {}
    }

    # Prepare TF-IDF for cluster keywords / prototype doc keywords
    text_field = None
    for candidate in ['Summary', 'full_text', 'text']:
        if candidate in df.columns:
            text_field = candidate
            break
    if text_field is None:
        raise ValueError("No text column found in df. Expected one of ['Summary','full_text','text'].")

    corpus = df[text_field].fillna("").astype(str).tolist()
    tfidf = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1,2))
    tfidf_matrix = tfidf.fit_transform(corpus)
    feature_names = np.array(tfidf.get_feature_names_out())

    # cluster centroids in reduced space (use mean of members)
    centroids = np.vstack([reduced_data[cluster_labels == c].mean(axis=0) for c in unique_clusters])

    # 3) For each cluster summarize SHAP importances (per PCA component) and map back to original embedding dims
    for class_idx, cluster_id in enumerate(unique_clusters):
        # shap_values[class_idx] shape (m, n_pcs)
        class_shap = shap_values[class_idx]  # sample x n_pcs
        mean_abs_shap = np.mean(np.abs(class_shap), axis=0)  # shape (n_pcs,)

        # Top PCA components for this cluster
        top_pc_idx = np.argsort(mean_abs_shap)[::-1][:top_pcs]
        top_pc_scores = mean_abs_shap[top_pc_idx]

        # Map PCA importance -> original embedding dims (approximate)
        # pca.components_ shape: (n_pcs, original_dim)
        # To get per-original-dim importance: aggregate absolute contribution across selected PCs
        # Use weighted sum of absolute PCA loadings by mean_abs_shap
        # result shape: (original_dim,)
        pca_components = pca_model.components_  # (n_pcs, orig_dim)
        # Weighted contribution from all PCs using the mean_abs_shap
        contribution_orig = np.abs((mean_abs_shap[:, None] * np.abs(pca_components))).sum(axis=0)
        top_embed_idx = np.argsort(contribution_orig)[::-1][:top_embeddings]
        top_embed_scores = contribution_orig[top_embed_idx]

        # 4) Prototype documents: nearest to centroid
        members_idx = np.where(cluster_labels == cluster_id)[0]
        if members_idx.size == 0:
            prototypes = []
        else:
            member_coords = reduced_data[members_idx]
            centroid = centroids[class_idx]
            dists = np.linalg.norm(member_coords - centroid[None, :], axis=1)
            nearest = np.argsort(dists)[:n_prototypes]
            prototypes = members_idx[nearest].tolist()

        # 5) Top TF-IDF keywords for the cluster (aggregate TF-IDF across members)
        if members_idx.size > 0:
            cluster_tfidf_sum = np.asarray(tfidf_matrix[members_idx].sum(axis=0)).ravel()
            top_terms_idx = np.argsort(cluster_tfidf_sum)[::-1][:20]
            top_terms = feature_names[top_terms_idx]
        else:
            top_terms = []

        # Store results
        results['cluster_summaries'][int(cluster_id)] = {
            'top_pca_components_idx': top_pc_idx.tolist(),
            'top_pca_component_scores': top_pc_scores.tolist(),
            'top_embedding_dims_idx': top_embed_idx.tolist(),
            'top_embedding_dim_scores': top_embed_scores.tolist(),
            'prototype_doc_indices': prototypes,
            'top_tfidf_terms': top_terms.tolist()
        }

        # Print short human-friendly summary
        print(f"\nCluster {cluster_id} — top PCA components: {top_pc_idx.tolist()} (scores ~ {top_pc_scores.round(4).tolist()})")
        print(f"Cluster {cluster_id} — prototype doc indices: {prototypes}")
        print(f"Cluster {cluster_id} — top TF-IDF terms: {top_terms[:10].tolist()}")

        # Optional: SHAP summary plot for this class (requires display)
        try:
            shap.summary_plot(class_shap, shap_X, feature_names=[f"PC{i+1}" for i in range(n_pcs)], show=True)
        except Exception:
            # In headless environments plotting may fail; ignore
            pass

    return results
